In [37]:
import os

In [38]:
%pwd

'd:\\Python\\End-To-End Ml Project\\End-to-end-ML-Project-with-MLflow'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file :Path
    unzip_dir: Path

In [40]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:

    def __init__(self,
                config_filepath = CONFIG_FILE_PATH, # reading config/config.yaml file 
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
         
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) # create artifacts folder

    
    # DataIngestionConfig - is custom entity(datatype) created for return purspose(custom return type)
    def get_data_ingestion_config(self)-> DataIngestionConfig: # this fun only reads config file for data ingestion

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir =config.unzip_dir
        )

        return data_ingestion_config

In [42]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size 

In [43]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info : \n {headers}")
        else:
            logger.info(f"file alreadt exists of size:{get_size(Path(self.config.local_data_file))}")   



    def extract_zip_file(self):
        
        unzip_path =self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [45]:
# read the data using pipeline(setp by step)

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2023-10-22 02:54:31,203: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 02:54:31,205: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 02:54:31,207: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-22 02:54:31,209: INFO: common: created directory at artifacts]
[2023-10-22 02:54:31,562: INFO: common: created directory at artifcats/data_ingestion]


BoxKeyError: "'ConfigBox' object has no attribute 'source_url'"